# Extract Sentinel-1 Data and Sentinel-2 data for a 5*5 pixels bounding box of given lat and long

In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd
import xarray as xr

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
#pc.settings.set_subscription_key('******')

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()
import os

In [ ]:
os.chdir('/home/jovyan/EY-Crop-identification')
print(os.getcwd())

/home/jovyan/EY-Crop-identification


In [ ]:
def get_sentinel_data(latlong, time_slice, assets, sen1_box_deg, sen2_box_deg, run_data_sen1=True, run_data_sen2=True):
    '''
    Returns sentinel-1-rt and sentinel-2-l2a for a bounding box of a given latitude and longitude 
    Attributes:
    latlong - A tuple with 2 elements - latitude and longitude
    time_slice - Timeframe for which the VV and VH values have to be extracted
    assets - A list of bands to be extracted
    sen1_box_deg - bounding box degree for sentinel-1-rt
    sen2_box_deg - bounding box degree for sentinel-2-l2a
    run_data_sen1 - Set to true for running sentinel-1-rt
    run_data_sen2 - Set to true for running sentinel-2-l2a
    '''

    latlong=latlong.replace('(','').replace(')','').replace(' ','').split(',')
    data_sen1, data_sen2 = None, None
    lat, long = float(latlong[0]), float(latlong[1])
    

    time_of_interest = time_slice

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    bands_of_interest = assests
    if run_data_sen1:
        min_lon = long-sen1_box_deg/2
        min_lat = lat-sen1_box_deg/2
        max_lon = long+sen1_box_deg/2
        max_lat = lat+sen1_box_deg/2
        bbox_of_interest = (min_lon, min_lat, max_lon, max_lat)
        
        search_sen1 = catalog.search(
            collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest
        )
        items_sen1 = list(search_sen1.get_all_items())
        data_sen1 = stac_load(items_sen1, patch_url=pc.sign, bbox=bbox_of_interest)
    
    if run_data_sen2:
        min_lon = long-sen2_box_deg/2
        min_lat = lat-sen2_box_deg/2
        max_lon = long+sen2_box_deg/2
        max_lat = lat+sen2_box_deg/2
        bbox_of_interest = (min_lon, min_lat, max_lon, max_lat)
        
        search_sen2 = catalog.search(collections=["sentinel-2-l2a"], bbox=bbox_of_interest, datetime=time_of_interest)

        items_sen2 = list(search_sen2.get_all_items())

        resolution = 10  # meters per pixel
        scale = resolution / 111320.0
        data_sen2 = stac_load(items_sen2,
                              bands=["red", "green", "blue", "nir", "SCL"],
                              resolution=scale,
                              bbox=bbox_of_interest,
                              crs="EPSG:4326",
                              patch_url=pc.sign)
    return data_sen1, data_sen2

In [ ]:
# defined bounding boxes
bboxes = [
    (0.00024, 0.00024),# 3*3
    (0.0004, 0.0004),# 5*5
    (0.00056, 0.00056)# 7*7
]
time_slice = "2021-11-01/2022-12-01"
assests = ['vh','vv']

def run_bbox(bboxes, box_num, time_slice, crop_presence_data, train, start=0, run_data_sen1=True, run_data_sen2=True):
    '''
    Extract and save Sentinel-1 and Sentinel-2 bounding box data for given latitude and longitude coordinates
    '''
    bbox = bboxes[box_num]
    j = start
    for coordinates in tqdm(crop_presence_data['Latitude and Longitude'][start:]):
        data_sen1, data_sen2 = get_sentinel_data(coordinates,time_slice,assests, bbox[0], bbox[1], run_data_sen1, run_data_sen2)
        if train:
            if data_sen1:
                data_sen1.to_netcdf(f"Data/bbox{box_num}/train/vhvv_bbox{box_num}_row{j}.nc")
            if data_sen2:
                data_sen2.to_netcdf(f"Data/bbox{box_num}/train/vhvv_bbox{box_num}_sen2_row{j}.nc")
        else:
            if data_sen1:
                data_sen1.to_netcdf(f"Data/bbox{box_num}/test/vhvv_bbox{box_num}_row{j}.nc")
            if data_sen2:
                data_sen2.to_netcdf(f"Data/bbox{box_num}/test/vhvv_bbox{box_num}_sen2_row{j}.nc")
        j += 1


## Run training data

In [ ]:
crop_presence_data = pd.read_csv("Data/Crop_Location_Data_20221201.csv")
train = True
box_num = 1

In [ ]:
run_bbox(bboxes, box_num, time_slice, crop_presence_data, train, start=43, run_data_sen1=False, run_data_sen2=True)

## Run training data

In [ ]:
crop_presence_data = pd.read_csv("Data/challenge_1_submission_template_correct_columns_fixed.csv")
train = False
box_num = 1

In [ ]:
run_bbox(bboxes, box_num, time_slice, crop_presence_data, train, start=38, run_data_sen1=True, run_data_sen2=True)

100%|██████████| 212/212 [2:08:50<00:00, 36.46s/it]  
